In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [29]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## Get Indeks Link

In [2]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="article__list clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
#         print(link)
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = el_page.findAll('div', class_='paging__item')[-1].find('a')
        if  el_page.findAll('div', class_='paging__item')[-1].find('a', class_="paging__link paging__link--active"):
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

## Get Detail Berita

In [21]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

        #category
        articles['category'] = link[1]
        articles['url'] = url

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text

        #article
        article = soup.find("div", class_="read__content")

        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

        #extract author
        articles['author'] = soup.find('div', class_="read__author").text

        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'kompas'

        #extract comments count
#         options = Options()
#         options.add_argument('--headless')
#         options.add_argument('--disable-gpu')  # Last I checked this was necessary.
#         options.add_argument('--disable-extensions')

#         driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
#         driver.get(url)
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html5lib')

#         comment = soup.find('div', class_="span4 comments-count tright")
#         comment_num = comment.text.replace('Ada ', '')
#         comment_num = comment_num.replace(' komentar untuk artikel ini', '')
        articles['comments'] = 0

        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = tags

        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']

        #hapus all script
        for script in article.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [8]:
links = getIndeksLink([], 1, 'news', 'news')

page  1
https://indeks.kompas.com/news/2018-07-31/1
page  2
https://indeks.kompas.com/news/2018-07-31/2
page  3
https://indeks.kompas.com/news/2018-07-31/3
page  4
https://indeks.kompas.com/news/2018-07-31/4
page  5
https://indeks.kompas.com/news/2018-07-31/5
page  6
https://indeks.kompas.com/news/2018-07-31/6
page  7
https://indeks.kompas.com/news/2018-07-31/7


In [31]:
details = getDetailBerita(links[0:2])
details

[{'author': 'Kontributor Palembang, Aji YK Putra',
  'category': 'news',
  'comments': 0,
  'content': ' PALEMBANG, KOMPAS.com -\xa0Aksi pencurian yang dilakukan JR (25 ) warga  Palembang, Sumatera Selatan tak berjalan mulus.Sebab, Romadhon malah tepergok  korban ketika hendak masuk ke dalam rumah.  Korban yang diketahui bernama Joni (37) pun langsung mencoba menangkap pelaku, namun, Romadhon yang membawa sebilah gunting, langsung menusukkannya ke perut Joni.Dikatakan Joni, kejadian itu bermula ketika istrinya melihat pelaku masuk dari tangga lantai tiga melewati pintu terali besi dari rekaman CCTV. Melihat hal tersebut, korban langsung mencoba mendekati pelaku."Dia bilang disuruh orang dibawa ambil barang, padahal dibawah sudah tutup dan tidak ada orang,” kata Joni, Senin (30/7/2018).Baca juga: Demi Fakultas Kedokteran, 8 Calon Mahasiswa Pakai Alat Canggih Penjawab SoalMerasa aksinya tepergok, Romadhon pun terdesak. Ia langsung menusuk Joni dengan menggunakan sebilah gunting. Karena t

In [27]:
df = pd.DataFrame(details)

In [28]:
df

,author,category,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,"Kontributor Palembang, Aji YK Putra",news,0,"PALEMBANG, KOMPAS.com - Aksi pencurian yang d...",1635060,https://asset.kompas.com/crop/89x7:1000x615/78...,2018-07-31 14:00:00,kompas,Regional,"tusuk, korban, pencuri, residivis, Palembang","Tepergok Korban Saat Beraksi, Residivis Pencur...",http://regional.kompas.com/read/2018/07/31/140...
1,Nursita Sari,news,0,"JAKARTA, KOMPAS.com - Jalur sepeda di trotoa...",1635296,https://asset.kompas.com/crop/56x0:900x563/780...,2018-07-31 13:50:16,kompas,Megapolitan,"jalur sepeda, trotoar, GBK",Jalur Sepeda di Trotoar Jalan Asia Afrika Kawa...,http://megapolitan.kompas.com/read/2018/07/31/...


In [446]:
soup.findAll('a', class_="inner-link")

[<a class="inner-link" href="http://indeks.kompas.com/tag/Bandar-Lampung" style="color:#428bca;" target="_blank">BANDAR LAMPUNG</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Ombudsman" style="color:#428bca;" target="_blank">Ombudsman</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Lampung-Selatan" style="color:#428bca;" target="_blank">Lampung Selatan</a>,
 <a class="inner-link" href="http://nasional.kompas.com/read/2018/07/28/18401441/kasus-suap-bupati-lampung-selatan-kpk-geledah-lima-tempat" target="_blank">Kasus Suap Bupati Lampung Selatan, KPK Geledah Lima Tempat</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/OTT" style="color:#428bca;" target="_blank">OTT</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/KPK" style="color:#428bca;" target="_blank">KPK</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Zainuddin-Hasan" style="color:#428bca;" target="_blank">Zainuddin Hasan</a>,
 <a class="inner-link" href="htt